<a href="https://colab.research.google.com/github/Fitzpatrique/AI-Chatbot/blob/main/CB_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
import numpy as np
import re
from nltk.stem import wordnet # to perform lemmitization
#from sklearn.feature_extraction.text import CountVectorizer # to perform bow
from sklearn.feature_extraction.text import TfidfVectorizer # to perform tfidf
from nltk import pos_tag # for parts of speech
from sklearn.metrics import pairwise_distances # to perform cosine similarity
from nltk import word_tokenize #  to create tokens
#from nltk.corpus import stopwords # for stop words

In [ ]:
df = pd.read_excel('C:\\Users\\USEER\\Desktop\\Patrick\\Project\\CB\\dialog_talk_agent.xlsx')
#df.head(5)

In [ ]:
df.ffill(axis = 0, inplace = True) # fills the null value with the previous value.
#df.head(5)

In [ ]:
def text_normalization(text):
    text = str(text).lower() # text to lower case
    spl_char_text = re.sub(r'[^a-z]',' ',text) # removing special characters
    tokens = nltk.word_tokenize(spl_char_text) # word tokenizing
    lema = wordnet.WordNetLemmatizer() # initializing lemmatization
    tags_list = pos_tag(tokens, tagset = None) # parts of speech
    lema_words = [] # empty list
    
    for token,pos_token in tags_list:
        if pos_token.startswith('V'): # verb
            pos_val = 'v'
        elif pos_token.startswith('J'): # adjective
            pos_val = 'a'
        elif pos_token.startswith('R'): # adverb
            pos_val = 'r'
        else:
            pos_val = 'n' # noun
            
        lema_token = lema.lemmatize(token, pos_val) # performing lemmmatization
        lema_words.append(lema_token) # appending lemmatized token into a list
    
    return " ".join(lema_words) # returns the lemmatized tokens as a sentence

In [ ]:
text_normalization('telling you stuff about me')

'tell you stuff about me'

In [ ]:
df['lemmatized_text'] = df['Context'].apply(text_normalization) # applying the function to dataset to get clean text
#df.head(5)

In [ ]:
# using tf-idf

tfidf = TfidfVectorizer() # initializing tf-idf
x_tfidf = tfidf.fit_transform(df['lemmatized_text']).toarray() # transforming the data to array

In [ ]:
df_tfidf = pd.DataFrame(x_tfidf, columns=tfidf.get_feature_names())
#df_tfidf.head(5)

In [ ]:
def response(text):
    lemma = text_normalization(text) # calling the function to perform text normalization
    tf = tfidf.transform([lemma]).toarray() # applying tf-idf
    cos = 1 - pairwise_distances(df_tfidf,tf,metric='cosine') # applying cosing similarity
    index_value = cos.argmax() # getting the index value
    return df['Text Response'].loc[index_value]

In [ ]:
def run():
    while True:
        text = str(input("Enter the 'q' key to exit conversation: "))
        if text == 'q':
            print("Bye!")
            return False
        else:
            print(response(text))
            return run()
        

In [ ]:
run()

Enter the 'q' key to exit conversation: hey
Hi there, friend!
Enter the 'q' key to exit conversation: how are you
Lovely, thanks.
Enter the 'q' key to exit conversation: fuck you too
Thanks! The feeling is mutual.
Enter the 'q' key to exit conversation: and your mamsa
Lovely, thanks.
Enter the 'q' key to exit conversation: bye
See you soon!
Enter the 'q' key to exit conversation: q
Bye!


False